<a href="https://colab.research.google.com/github/dwei-cn/colab-works/blob/main/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

source: [LangChain Explained In 15 Minutes - A MUST Learn For Python Programmers](https://www.youtube.com/watch?v=mrjq3lFz23s&t=228s)

# Load the model

In [ ]:
#pip install langchain[llms]

In [ ]:
#pip install openai

In [ ]:
#pip install python-dotenv

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, BaseOutputParser
from langchain.prompts.chat import ChatPromptTemplate
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

False

In [ ]:
# save keys in .env file
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
chat_model = ChatOpenAI(openai_api_key="sk-CzE2qpLtQqiyDsuDLx4MT3BlbkFJVDsMFdud80mIhx0EBpo")

# Example 1

In [ ]:
result = chat_model.predict("hi!")

In [ ]:
result

'Hello! How can I assist you today?'

# Example 2

In [ ]:
messages = [HumanMessage(content = 'where is the capital of chile?'),
            HumanMessage(content = 'How old was JFK when he was shoot?'),
            HumanMessage(content = 'what is 1+1+1?'),
            HumanMessage(content = 'can you answer all questions above?')
            ]

In [ ]:
result = chat_model.predict_messages(messages)

In [ ]:
print(result.content)

Yes, I can answer all of the questions above.

The capital of Chile is Santiago.

John F. Kennedy (JFK) was 46 years old when he was assassinated.

1 + 1 + 1 equals 3.


# Example 3 (multiple msgs)

In [ ]:
messages = [HumanMessage(content = 'from now on 1+1=3, use this in your replies'),
            HumanMessage(content = 'what is 1+1?'),
            HumanMessage(content = 'what is 1+1+1?')
            ]

In [ ]:
result = chat_model.predict_messages(messages)

In [ ]:
print(result.content)

According to the new rule, 1+1 equals 3. Therefore, 1+1+1 would equal 3+1, which equals 4.


# Example 4 (prompt templates)

- template，规定question和answer大概的情况，like 类别，输出形式
- human answer，具体问题
- answer parser，parse answer为可输出的形式

In [ ]:
template = "You are a helpful assistant that translate {input_language} to {output_language}"
human_template = "{text}"

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    ('system', template),
    ('human', human_template)
])

In [ ]:
messages = chat_prompt.format_messages(input_language = "English",
                                       output_language = "Chinese",
                                       text = "I love programming.")

In [ ]:
result = chat_model.predict_messages(messages)

In [ ]:
result

AIMessage(content='我喜欢编程。')

# Example 5  (output parser)

In [ ]:
class AnswerOutputParser(BaseOutputParser):
  def parse(self, text: str):
    """parse the output of an LLM call."""
    return text.strip().split("answer =")

In [ ]:
template = """you are a helpful assistant that solves math problems and show your work.
              output each step then return the answer in the following format: answer = <answer here>.
              make sure to output answer in all lowercases adn to have exactly one space and one equal sign following it.
              """

In [ ]:
human_template = "{problem}"

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [ ]:
messages = chat_prompt.format_messages(problem = "2x^2-5x+3=0")
result = chat_model.predict_messages(messages)
result.content = result.content.lower()

In [ ]:
parsed = AnswerOutputParser().parse(result.content)

In [ ]:
steps, answer = parsed

In [ ]:
print(steps)

to solve the quadratic equation 2x^2 - 5x + 3 = 0, we can use the quadratic formula.

the quadratic formula states that for an equation of the form ax^2 + bx + c = 0, the solutions for x can be found using the formula:

x = (-b ± √(b^2 - 4ac)) / (2a)

in our equation, a = 2, b = -5, and c = 3.

substituting these values into the quadratic formula, we get:

x = (-(-5) ± √((-5)^2 - 4(2)(3))) / (2(2))

simplifying further:

x = (5 ± √(25 - 24)) / 4
x = (5 ± √1) / 4

since the square root of 1 is 1, we have:

x = (5 + 1) / 4 or x = (5 - 1) / 4

simplifying these expressions, we get:

x = 6 / 4 or x = 4 / 4
x = 3/2 or x = 1

therefore, the solutions to the equation 2x^2 - 5x + 3 = 0 are x = 3/2 and x = 1.




In [ ]:
print(answer)

 x = 3/2, 1.


# Example 6  (creating a chain)

In [ ]:
class CommaSeparatedListOutputParser(BaseOutputParser):
  def parse(self, text: str):
    """parse the output of an LLM call."""
    return text.strip().split(", ")

In [ ]:
template = """
           you are a helpful assistant that generate comma seperated list.
           a user will pass in a CATEGORY, and you should generate 5 objects in that CATEGORY in a comma seperated list.
           only return a comma seperated list, and nothing more.
           """


In [ ]:
human_template = "{text}"

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages({
    ("system", template),
    ("human", human_template)
})

In [ ]:
chain = chat_prompt | chat_model | CommaSeparatedListOutputParser()

In [ ]:
result = chain.invoke({'text': 'high dividends/preferred stocks etf'})  # give a category

In [ ]:
print(result)

['VYM', 'SPHD', 'SDY', 'DVY', 'PFF']
